In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
import zipfile
import os

zip_path = '/content/drive/MyDrive/MLDL_repo/GTA5.zip'
extract_path = '/content/dataset'

os.makedirs(extract_path, exist_ok=True)

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print("Extraction complete!")

Extraction complete!


In [3]:
import zipfile
import os

zip_path = '/content/drive/MyDrive/MLDL_repo/Cityscapes.zip'
extract_path = '/content/cityscapes'

os.makedirs(extract_path, exist_ok=True)

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print("Extraction complete!")

Extraction complete!


In [4]:
import sys
sys.path.append('/content/drive/MyDrive/MLDL_repo/step3b')

In [5]:
!pip install -U fvcore

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 5.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 4.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for fvcore: filename=fvcore-0.1.5.post20221221-py3-none-any.whl size=61397 sha256=56ffad16cd7fc58c935fe57aaac2388c02da646533a8466f9a41d38cbb8b13ce
  Stored in directory: /root/.cache/pip/wheels/65/71/95/3b8fde5c65c6e4a806e0867c1651dcc71a1cb2f3430e8f355f
  Created wheel for iopath: filename=iopath-0.1.10-py3-none-any.whl size=31527 sha256=15ede69af8d6ccc7309156df472395527a9082869e1bae66573d72d9d0b99ffd
  Stored in directory: /root/.cache/pip/wheels/ba/5e/16/6117f8fe7e9c0c161a795e10d94645ebcf301ccbd01f66d8ec
Successfully built fvcore iopath


In [ ]:
import re
import os
import re

def find_latest_checkpoint(checkpoint_dir):
    if not os.path.exists(checkpoint_dir):
        print(f"Checkpoint directory {checkpoint_dir} does not exist.")
        return None

    checkpoints = [f for f in os.listdir(checkpoint_dir) if f.startswith("bisenet_epoch_") and f.endswith(".pt")]
    if not checkpoints:
        print(f"No checkpoints found in {checkpoint_dir}.")
        return None

    def extract_epoch(fname):
        match = re.search(r"bisenet_epoch_(\d+).pt", fname)
        return int(match.group(1)) if match else -1

    checkpoints.sort(key=extract_epoch, reverse=True)

    latest = os.path.join(checkpoint_dir, checkpoints[0])
    print(f" Found latest checkpoint: {latest}")
    return latest

In [ ]:
import albumentations as A
from albumentations.pytorch import ToTensorV2
from torchvision import transforms

def get_train_transform():
    return A.Compose([
        A.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.05, p=0.5),
        A.Resize(720, 1280),
        A.Normalize(mean=(0.485, 0.456, 0.406),
                    std=(0.229, 0.224, 0.225)),
        ToTensorV2()
    ])


def get_cityscapes_val_transform():
    return transforms.Compose([
        transforms.Resize((512, 1024)),
        transforms.ToTensor()
    ])

def get_cityscapes_target_transform():
    return transforms.Compose([
        transforms.Resize((512, 1024), interpolation=transforms.InterpolationMode.NEAREST),
        transforms.PILToTensor()
    ])


In [ ]:
import os
import torch
from torchvision import transforms
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
from torch.utils.data import Subset
from torchvision import transforms as T
from PIL import Image
from datasets_custom.gta5_aug import GTA5
from models.bisenet.build_bisenet import BiSeNet
from train import train_one_epoch
from train import validate

def main():

    dataset_root = '/content/dataset/GTA5'

    # Create the full dataset without transforms first
    full_dataset = GTA5(root=dataset_root)

    # Then split
    indices = list(range(len(full_dataset)))
    train_indices, val_indices = train_test_split(indices, test_size=0.30, random_state=42)

    # Create two GTA5 datasets with different transforms
    train_dataset = GTA5(root=dataset_root, transform=get_train_transform())
    val_dataset = GTA5(root=dataset_root, transform=get_cityscapes_target_transform())

    # Subset the datasets
    train_dataset = Subset(train_dataset, train_indices)
    val_dataset = Subset(val_dataset, val_indices)
    print(f"Train dataset size: {len(train_dataset)}")
    print(f"Val dataset size: {len(val_dataset)}")

    train_loader = DataLoader(train_dataset, batch_size=2, shuffle=True, num_workers=2)
    val_loader = DataLoader(val_dataset, batch_size=2, shuffle=False, num_workers=2)

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    num_classes = 19
    base_lr = 2.5e-4
    batch_size = 2
    epochs = 50
    context_path = 'resnet18'
    checkpoint_dir = "/content/drive/MyDrive//MLDL_step3b/checkpoints_3b_CJ"

    if not os.path.exists(checkpoint_dir):
      os.makedirs(checkpoint_dir)


    model = BiSeNet(num_classes=num_classes, context_path=context_path)
    if torch.cuda.device_count() > 1:
        print("Using", torch.cuda.device_count(), "GPUs")
        model = nn.DataParallel(model)

    model = model.to(device)
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    start_epoch = 0
    start_batch = 0

    latest_ckpt = find_latest_checkpoint(checkpoint_dir)
    print("latest_ckpt is:", latest_ckpt)
    latest_ckpt = find_latest_checkpoint(checkpoint_dir)
    print("latest_ckpt is:", latest_ckpt)

    if latest_ckpt:
        print(f"Restore from checkpoint: {latest_ckpt}")
        checkpoint = torch.load(latest_ckpt, map_location=device)
        model.load_state_dict(checkpoint['model_state_dict'])
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        start_epoch = checkpoint['epoch'] + 1
        start_batch = 0  # Non serve più riprendere dal batch
        print(f"Picking up from epoch {start_epoch}")
    else:
        print("No checkpoint found, start training from scratch")

    # Check Data and Labels
    for images, labels in train_loader:
        print("Images shape:", images.shape, "dtype:", images.dtype)
        print("Labels shape:", labels.shape, "dtype:", labels.dtype)
        print("Unique labels:", torch.unique(labels))
        break  # Print for the first batch only

    # Training
    for epoch in range(start_epoch, epochs):
        current_start_batch = start_batch if epoch == start_epoch else 0
        train_one_epoch(model, train_loader, optimizer, base_lr, epoch, epochs, device,
                        checkpoint_dir=checkpoint_dir, start_batch=current_start_batch)

    final_model_path = "/content/drive/MyDrive/MLDL_step3b/checkpoints_3b_CJ/bisenet_final_3b_CJ.pt"
    torch.save(model.state_dict(), final_model_path)
    print(f"Modello finale salvato in: {final_model_path}")



if __name__ == "__main__":
    main()


Train dataset size: 1750
Val dataset size: 750


Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 141MB/s]
Downloading: "https://download.pytorch.org/models/resnet101-63fe2227.pth" to /root/.cache/torch/hub/checkpoints/resnet101-63fe2227.pth
100%|██████████| 171M/171M [00:01<00:00, 177MB/s]


Output streaming troncato alle ultime 5000 righe.
📉 Loss: 0.08147339522838593

 Epoch 49/50 - Batch 87/875
📉 Loss: 0.08717609941959381

 Epoch 49/50 - Batch 88/875
📉 Loss: 0.05648065730929375

 Epoch 49/50 - Batch 89/875
📉 Loss: 0.07694459706544876

 Epoch 49/50 - Batch 90/875
📉 Loss: 0.06288690119981766

 Epoch 49/50 - Batch 91/875
📉 Loss: 0.07502006739377975

 Epoch 49/50 - Batch 92/875
📉 Loss: 0.047480423003435135

 Epoch 49/50 - Batch 93/875
📉 Loss: 0.0803271010518074

 Epoch 49/50 - Batch 94/875
📉 Loss: 0.04893811047077179

 Epoch 49/50 - Batch 95/875
📉 Loss: 0.07468455284833908

 Epoch 49/50 - Batch 96/875
📉 Loss: 0.07701677083969116

 Epoch 49/50 - Batch 97/875
📉 Loss: 0.0518488809466362

 Epoch 49/50 - Batch 98/875
📉 Loss: 0.06182039901614189

 Epoch 49/50 - Batch 99/875
📉 Loss: 0.0802631825208664

 Epoch 49/50 - Batch 100/875
📉 Loss: 0.04268769174814224

 Epoch 49/50 - Batch 101/875
📉 Loss: 0.0739636942744255

 Epoch 49/50 - Batch 102/875
📉 Loss: 0.06074940785765648

 Epoch 49

In [8]:
import os
import torch
from torchvision import transforms
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
from torch.utils.data import Subset
from torchvision import transforms as T
from PIL import Image
from datasets_custom.gta5_aug import GTA5
from datasets_custom.cityscapes import CityScapes
from models.bisenet.build_bisenet import BiSeNet
from datasets_custom.labels import GTA5Labels_TaskCV2017
from train import validate


#initialize model
num_classes = 19
context_path = 'resnet18'


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = BiSeNet(num_classes=num_classes, context_path=context_path).to(device)


#load model
model_path = r'/content/drive/MyDrive/MLDL_repo/step3b/final_models/bisenet_final_3b_CJ-2.pt'
model.load_state_dict(torch.load(model_path, map_location='cpu'), strict = False )

model.eval() #python built in function

# Dataset definitions and transformation
transform = transforms.Compose([
        transforms.Resize((512, 1024)),
        transforms.ToTensor(),
        transforms.Normalize(mean=(0.485, 0.456, 0.406),
                    std=(0.229, 0.224, 0.225)),
    ])

target_transform = transforms.Compose([
        transforms.Resize((512, 1024), interpolation=transforms.InterpolationMode.NEAREST),
        transforms.PILToTensor()
    ])

dataset_root = '/content/cityscapes/Cityscapes/Cityspaces'

test_dataset = CityScapes(root=dataset_root, split='val', transform=transform, target_transform=target_transform)
print(f"Test dataset size: {len(test_dataset)}")
test_loader = DataLoader(test_dataset, batch_size=2, shuffle=True, num_workers=2)


# results
best_miou, miou, per_class_ious = validate(model, test_loader, num_classes, device, best_miou=0.0)
print("\n Validation Results on Cityscape:")
print(f" - mIoU: {miou:.4f}")
for idx, label in enumerate(GTA5Labels_TaskCV2017.list_):
    print(f"{label.name:>15}: IoU = {per_class_ious[idx]:.4f}")

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 194MB/s]
Downloading: "https://download.pytorch.org/models/resnet101-63fe2227.pth" to /root/.cache/torch/hub/checkpoints/resnet101-63fe2227.pth
100%|██████████| 171M/171M [00:01<00:00, 116MB/s]


Loaded 500 images for split: val
Test dataset size: 500


Validating: 100%|██████████| 250/250 [00:58<00:00,  4.29it/s]

 Validation mIoU: 0.2365
 New best mIoU found!

 Validation Results on Cityscape:
 - mIoU: 0.2365
           road: IoU = 0.7088
       sidewalk: IoU = 0.1943
       building: IoU = 0.6804
           wall: IoU = 0.1439
          fence: IoU = 0.0228
           pole: IoU = 0.0001
          light: IoU = 0.0082
           sign: IoU = 0.0039
     vegetation: IoU = 0.7720
        terrain: IoU = 0.2585
            sky: IoU = 0.7878
         person: IoU = 0.3270
          rider: IoU = 0.0104
            car: IoU = 0.4891
          truck: IoU = 0.0437
            bus: IoU = 0.0231
          train: IoU = 0.0045
      motocycle: IoU = 0.0150
        bicycle: IoU = 0.0000
